In [1]:
import os
import mxnet as mx
import pandas as pd
import cv2
import numpy as np
import lightgbm as lgb

ImportError: No module named mxnet

In [ ]:
curpath = "E:/train/"
os.chdir(curpath)

In [ ]:
img_lst = pd.read_csv('./model_train.lst', sep = '\t', header = None, names =('imgrank','type','path'))

In [ ]:
img_lst.shape[0]

In [ ]:
def get_image(filename):
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # change bgr to rgb
    img = cv2.resize(img, (224,224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]  # change to example,channel,width,height
    return img

In [ ]:
# 提取中间层的输出进行训练模型；
def feature_extract(prefix, epoch):
    sym, arg_params, aux_params = mx.model.load_checkpoint(prefix, epoch)
    internals = sym.get_internals()
    fea_symbol = internals["fc1_output"]
    return(fea_symbol, arg_params, aux_params)

In [ ]:
fea_model, arg_params, aux_params = feature_extract('D:/gitcode/DataScience/Python/python-mxnet-notebook/how_to/resnet-50',0)

In [ ]:
# reset model
mod =mx.mod.Module(symbol=fea_model, context=mx.cpu())
mod.bind(for_training = False, data_shapes = [('data',(1,3,224,224))])  # should set the same with before
mod.set_params(arg_params, aux_params)

In [ ]:
# define model 
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

In [ ]:
len(img_lst)
i=1
curpath+img_lst['path'][i]
img = get_image(curpath+img_lst['path'][i])
img.shape

In [ ]:
# 获取训练结合的结果
result = np.zeros((len(img_lst), 1000))
for i in range(1,len(img_lst)):
    print(curpath+img_lst['path'][i])
    img = get_image(curpath+img_lst['path'][i])
    mod.forward(Batch([mx.nd.array(img)]))
    result[i,:] = mod.get_outputs()[0].asnumpy()

In [ ]:
# train model ,x= result, y =img_lst['type]
# import sklearn 
# import xgboost as xgb

In [ ]:
# 获取测试集合
img_test = pd.read_csv('./model_test.lst', sep = '\t', header = None, names =('imgrank','type','path'))
testresult = np.zeros((len(img_test),1000))
for i in range(1,len(img_test)):
    print("num_%i:%s" %(i,curpath+img_test['path'][i]))
    img = get_image(curpath+img_test['path'][i])
    mod.forward(Batch([mx.nd.array(img)]))
    testresult[i,:] = mod.get_outputs()[0].asnumpy()

In [13]:
from sklearn import svm

In [14]:
x_train = result
y_train = img_lst['type']
x_test = testresult

NameError: name 'testresult' is not defined

In [ ]:
clf = svm.SVC(C=10, kernel='rbf', degree=3, coef0=0.0, shrinking=True, probability=True, tol=0.001, cache_size=200, 
                  class_weight=None, verbose=False, max_iter=-1, random_state=None)
clf.fit(x_train, y_train)

In [ ]:
# randomforest
from sklearn.ensemble import RandomForestClassifier
for i in range(10,300,10):
    rf = RandomForestClassifier(max_depth=5, n_estimators=i)
    rf.fit(x_train, y_train)
    ypred_rf = rf.predict(x_test)
    predict_temp = np.array(img_test['type']) - np.array(ypred_rf)
    count_num = np.count_nonzero(predict_temp)
    accuracy_rf = (len(predict_temp)-count_num)*1.0 / (len(predict_temp))
    print ('the accuracy of RandomForest in n_estimators  %i is %.4f' %(i,accuracy_rf))

In [ ]:
# ET
from sklearn.ensemble import ExtraTreesClassifier
for i in range(10,300,10):
    forest = ExtraTreesClassifier(n_estimators=i)
    forest.fit(x_train, y_train)
    ypred_rf = forest.predict(x_test)
    predict_temp = np.array(img_test['type']) - np.array(ypred_rf)
    count_num = np.count_nonzero(predict_temp)
    accuracy_rf = (len(predict_temp)-count_num)*1.0 / (len(predict_temp))
    print ('the accuracy of ExtraTreesClassifier in n_estimators  %i is %.4f' %(i,accuracy_rf))

In [ ]:
importance = forest.feature_importances_
importance

In [ ]:
# GBDT
from sklearn.ensemble import GradientBoostingClassifier
for i in range(10,300,10):
    gbdt = GradientBoostingClassifier(learning_rate=0.05, subsample=0.7, max_depth=6, n_estimators=i)
    gbdt.fit(x_train, y_train)
    ypred_rf = gbdt.predict(x_test)
    predict_temp = np.array(img_test['type']) - np.array(ypred_rf)
    count_num = np.count_nonzero(predict_temp)
    accuracy_rf = (len(predict_temp)-count_num)*1.0 / (len(predict_temp))
    print ('the accuracy of GradientBoostingClassifier in n_estimators  %i is %.4f' %(i,accuracy_rf))

In [ ]:
ypred_rf[1:100]

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(x_test)
predict_temp = np.array(img_test['type']) - np.array(y_pred)
count_num = np.count_nonzero(predict_temp)
accuracy_svm = (len(predict_temp)-count_num) / (len(predict_temp)/1.0)
proba_pred_svm = clf.predict_proba(x_test)
cm_svm = confusion_matrix(img_test['type'], y_pred)
print ('the accuracy of SVM is %.4f' %accuracy_svm)
print(cm_svm)

In [ ]:
# use lightgbm
# import lightgbm as lgb